In [ ]:
from PIL import Image
import os, glob
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

You can obtain rock/scissor/paper data by using [Google Teachable Machine](https://teachablemachine.withgoogle.com/)

In [ ]:
image_dir_path = os.getenv("HOME") + "/rock_scissor_paper/data/paper/"
image_pil = Image.open(image_dir_path + "papers1.jpg")  
image = np.array(image_pil)

plt.imshow(image)  
plt.show()

In [ ]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = os.getenv("HOME") + "/rock_scissor_paper/data/scissor"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/rock_scissor_paper/data/rock"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/rock_scissor_paper/data/paper"
resize_images(image_dir_path)

In [ ]:
def load_data(img_path, number_of_data=3180):  
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img   
        labels[idx]=0   
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=1   
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=2   
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 수: ", idx,)
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/rock_scissor_paper/data/"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

In [ ]:
import tensorflow as tf
from tensorflow import keras

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=20)

In [ ]:
def load_test(img_path, number_of_data=300):  
    img_size=28
    color=3
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=0   # Scissor : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=1   # Rock : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=2   # Paper : 2
        idx=idx+1
        
    print("테스트데이터(x_test)의 이미지 수: ",idx)
    return imgs, labels

In [ ]:
test_dir_path = os.getenv("HOME") + "/rock_scissor_paper/data/test/"   

resize_images(os.path.join(test_dir_path, "rock"))
resize_images(os.path.join(test_dir_path, "scissor"))
resize_images(os.path.join(test_dir_path, "paper"))

(x_test, y_test)=load_test(test_dir_path)   
x_test_norm = x_test/255.0   

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

In [ ]:
model.evaluate(x_test_norm, y_test)